In [3]:
import requests
import time
import tqdm
import pandas as pd
import xmltodict

### Gather data

In [5]:

delta_data = 200
t_wait = 0.5 # waiting time
data_total = []

# Init
url_bielfeld = 'http://pub.uni-bielefeld.de/oai?verb=ListRecords&metadataPrefix=oai_datacite'
x = requests.get(url_bielfeld)
o = xmltodict.parse(x.content)

data = o['OAI-PMH']['ListRecords']['record'] 
r_token = o['OAI-PMH']['ListRecords']['resumptionToken']['#text']
[data_total.append(d) for d in data]

while True:
    url_bielfeld = 'https://pub.uni-bielefeld.de/oai?verb=ListRecords&resumptionToken='+r_token
    x = requests.get(url_bielfeld)
    o = xmltodict.parse(x.content)
    data = o['OAI-PMH']['ListRecords']['record'] 
    if not o['OAI-PMH']['ListRecords']['resumptionToken'].get('#text', False):
        print('data loaded')
        break
    r_token = o['OAI-PMH']['ListRecords']['resumptionToken']['#text']
    [data_total.append(d) for d in data]
    if len(data_total) % 1000 == 0:
        print(len(data_total))
    time.sleep(t_wait)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
data loaded


### Functions//getter

In [17]:

def get_identifier(d):
    return d['header']['identifier']

def get_timestamps(d):
    return d['header']['datestamp']

def get_title(d):
    if type(d['metadata']['resource']['titles']['title'])!=list:
        return d['metadata']['resource']['titles']['title'].lower()
    else:
        return d['metadata']['resource']['titles']['title'][1]['#text'].lower()

def get_subjects(d):
    if d['metadata']['resource'].get('subjects', False):
        return [s.lower() for s in d['metadata']['resource']['subjects']['subject'] if type(s)==str]  
    else: 
        return []
    
def get_orci_authors(d):
    if type(d['metadata']['resource']['creators']['creator']) == list:
        return [{'orcid': c['nameIdentifier']['#text'].lower()} for c in d['metadata']['resource']['creators']['creator'] if c.get('nameIdentifier', False)]
    else:
        return [{'orcid': d['metadata']['resource']['creators']['creator']['nameIdentifier']['#text'].lower() if d['metadata']['resource']['creators']['creator'].get('nameIdentifier', False) else []}] 
                 
def get_authors(d):
    if type(d['metadata']['resource']['creators']['creator']) == list:
        return [{'givenName':c['givenName'].lower(), 'familyName': c['familyName'].lower()} for c in
                d['metadata']['resource']['creators']['creator']]
    else:
        return [{'givenName': d['metadata']['resource']['creators']['creator']['givenName'],
                 'familyName': d['metadata']['resource']['creators']['creator']['familyName']}]

def get_typology(d):
    return d['header']['setSpec']
    
def get_data(d, label):
    try: 
        if label=='identifier':
            return get_identifier(d)
        elif label=='timestamp':
            return get_timestamps(d)
        elif label=='authors':
            return get_authors(d)
        elif label=='orci_authors':
            return get_orci_authors(d)
        elif label=='title':
            return get_title(d)
        elif label=='subject_list': 
            return get_subjects(d)
        elif label=='typology':
            return get_typology(d)
    except:
        return 'error::'+label

    
def get_data_dict(d, ks_data):    
    return dict([(k,get_data(d,k)) for k in ks_data])


### Process data

In [20]:

ks_data = ['identifier','timestamp','authors','orci_authors','title','subject_list','typology']

processed_data = []
for i in tqdm.tqdm(range(len(data_total))):
    d = data_total[i]
    processed_data.append(get_data_dict(d, ks_data))


100%|██████████| 74200/74200 [00:00<00:00, 74651.53it/s]


### Build Dataframe

In [21]:
df = pd.DataFrame(dict([(k,[d[k] for d in processed_data]) for k in ks_data]))
df

,identifier,timestamp,authors,orci_authors,title,subject_list,typology
0,oai:pub.uni-bielefeld.de:1648257,2020-10-27T12:41:58Z,"[{'givenName': 'achim', 'familyName': 'bachem'...",[],five variations on a theme by gyula farkas,[],"[journal_article, doc-type:article]"
1,oai:pub.uni-bielefeld.de:1648277,2018-07-24T12:58:19Z,"[{'givenName': 'Rüdiger', 'familyName': 'Bittn...",[{'orcid': []}],is it reasonable to regret things one did?,[],"[journal_article, doc-type:article]"
2,oai:pub.uni-bielefeld.de:1648283,2018-07-24T12:58:53Z,"[{'givenName': 'k', 'familyName': 'knuttel'}, ...",[{'orcid': '0000-0003-0117-4021'}],(model) studies on vanadate-dependent bromo io...,"[v-51 nuclear, electron paramagnetic resonance...","[journal_article, doc-type:article]"
3,oai:pub.uni-bielefeld.de:1648289,2020-10-27T09:37:20Z,"[{'givenName': 'andreas', 'familyName': 'dress...",[],convex tree realizations of partitions,[],"[journal_article, doc-type:article]"
4,oai:pub.uni-bielefeld.de:1648292,2018-07-24T12:59:12Z,"[{'givenName': 'farhad', 'familyName': 'faisal...",[],prediction of a new sequence of peaks in above...,[],"[journal_article, doc-type:article]"
...,...,...,...,...,...,...,...
74195,oai:pub.uni-bielefeld.de:2383536,2021-05-18T15:39:51Z,"[{'givenName': 'michael', 'familyName': 'becks...",[{'orcid': '0000-0002-4405-3847'}],genlight: interactive high-throughput sequence...,[],"[conference, doc-type:conferenceObject]"
74196,oai:pub.uni-bielefeld.de:2949776,2021-05-18T15:54:36Z,"[{'givenName': 'renate', 'familyName': 'schüss...",[],das bildungssystem perus unter besonderer berü...,[],"[book_chapter, doc-type:bookPart]"
74197,oai:pub.uni-bielefeld.de:2954877,2021-05-18T15:54:19Z,"[{'givenName': 'renate', 'familyName': 'schüss...",[],fortbildungsangebote zur unterstützung forsche...,[],"[book_chapter, doc-type:bookPart]"
74198,oai:pub.uni-bielefeld.de:2953330,2021-05-18T15:59:07Z,"[{'givenName': 'milan', 'familyName': 'dransma...",[{'orcid': '0000-0002-9868-3596'}],achtsames laufen,[],"[journal_article, doc-type:article]"
